In [2]:
import pandas as pd
import numpy as np
from classifier import *
from dataloader import *

In [4]:
import multiprocessing as mp

In [6]:
df = get_dataset(data_files, True).drop_duplicates(['TC','TM','D'])
df.head()

,TC,TM,CUT,D,L,ABS,INT,JDK,ICB,DEP,...,TUAPI,UINT,SYNC,CALLSITES,AFPR,RBFA,EXPCAT,CONDCALL,PROJ,IS_MOCK
0,org.apache.hadoop.resourceestimator.translator...,testGetContainerSpec,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
1,org.apache.hadoop.resourceestimator.translator...,testGetJobSize,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
2,org.apache.hadoop.resourceestimator.translator...,testGetRecurrenceeId,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
3,org.apache.hadoop.resourceestimator.translator...,testStringToUnixTimestamp,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False
4,org.apache.hadoop.resourceestimator.translator...,testResourceSkyline,org.apache.hadoop.resourceestimator.translator...,org.apache.hadoop.yarn.api.records.Resource,real,True,False,False,False,0.5,...,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,Hadoop,False


In [8]:
df['TC'].count()

543171

In [10]:
pd.DataFrame([
    {
        'PRIOJ': p,
        'TOTAL': df[(df['PROJ']==p)]['PROJ'].count(),
        'MOCK': df[(df['PROJ']==p) & df['IS_MOCK']]['PROJ'].count(),

    }
    for p in projects
])

,PRIOJ,TOTAL,MOCK
0,Hadoop,323078,13484
1,Flink,86141,6719
2,Hive,23368,1490
3,Camel,12936,1855
4,CXF,22550,1489
5,Druid,45332,1869
6,HBase,11966,1072
7,Dubbo,8623,758
8,Oozie,5539,278
9,Storm,3638,369


In [12]:
("%d mocks(%.2f%%)" % (df['IS_MOCK'].sum(), df['IS_MOCK'].sum()/df['IS_MOCK'].count()*100))

'29383 mocks(5.41%)'

In [14]:
training_data = df.copy()

# Cross Project Prediction

In [16]:
def _run_core(par):
    from sklearn.feature_selection import SelectKBest, chi2
    p9 = par[0]
    p1 = par[1]
    train = balance_dataset(p9)
    test = balance_dataset(p1)
    X_train = train.drop(['CUT', 'IS_MOCK', 'TC', 'TM', 'D', 'L', 'PROJ'], axis=1)
    y_train = train['IS_MOCK']
    # feature_selection = SelectKBest(score_func=chi2, k=12)
    # X_train = feature_selection.fit_transform(X_train, y_train)
    X_test = test.drop(['CUT', 'IS_MOCK', 'TC', 'TM', 'D', 'L', 'PROJ'], axis=1)
    y_test = test['IS_MOCK']
    # X_test = feature_selection.transform(X_test)
    return run_classifier(X_train, X_test, y_train, y_test)

In [18]:
pl = mp.Pool()

def pool_core_bal(par):
    iter_count = 100
    project = par
    p9 = training_data[training_data['PROJ'] != project]
    p1 = training_data[training_data['PROJ'] == project]
    scores = pl.map_async(_run_core, ((p9, p1) for i in range(iter_count)))
    return project, scores

runing_result = [pool_core_bal(p) for p in projects]
runing_result = [
    (p,{
        k: np.mean([s[k] for s in r.get()])*100
        for k in metrics
    })
    for p,r in runing_result
]
pl.close()

In [20]:
perf_df = pd.DataFrame([
    {
        'project': proj,
        **perf
    }
    for proj, perf in runing_result
]).set_index(['project'])

In [22]:
perf_df

,accuracy,precision,recall,f1-score
project,,,,
Hadoop,75.081578,73.060139,79.464551,76.127886
Flink,75.388302,69.758053,89.636553,78.457752
Hive,65.903356,74.602175,48.212081,58.570688
Camel,67.110512,66.506446,68.944474,67.697884
CXF,71.555406,73.457337,67.546004,70.365373
Druid,66.503210,68.374688,61.426431,64.706740
HBase,76.419310,75.002291,79.262127,77.072561
Dubbo,66.678760,72.217085,54.323219,61.991363
Oozie,73.922662,80.716945,62.931655,70.676111


In [24]:
perf_plot = [
    {
        'project': p,
        'metric': m,
        'value': perf_df.at[p, m],
    }
    for p in projects
    for m in metrics
]
perf_plot = pd.DataFrame(perf_plot)

In [26]:
perf_df.describe()

,accuracy,precision,recall,f1-score
count,10.000000,10.000000,10.000000,10.000000
mean,70.971892,72.655295,67.922677,69.572653
std,4.138576,3.942011,12.361396,6.548617
min,65.903356,66.506446,48.212081,58.570688
25%,66.786698,70.372811,61.802737,65.454526
50%,71.355616,72.958964,67.512839,70.212774
75%,74.791849,74.315965,76.682714,74.764942
max,76.419310,80.716945,89.636553,78.457752


In [27]:
print(perf_df.round(2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  accuracy &  precision &  recall &  f1-score \\
project &           &            &         &           \\
\midrule
Hadoop  &     75.08 &      73.06 &   79.46 &     76.13 \\
Flink   &     75.39 &      69.76 &   89.64 &     78.46 \\
Hive    &     65.90 &      74.60 &   48.21 &     58.57 \\
Camel   &     67.11 &      66.51 &   68.94 &     67.70 \\
CXF     &     71.56 &      73.46 &   67.55 &     70.37 \\
Druid   &     66.50 &      68.37 &   61.43 &     64.71 \\
HBase   &     76.42 &      75.00 &   79.26 &     77.07 \\
Dubbo   &     66.68 &      72.22 &   54.32 &     61.99 \\
Oozie   &     73.92 &      80.72 &   62.93 &     70.68 \\
Storm   &     71.16 &      72.86 &   67.48 &     70.06 \\
\bottomrule
\end{tabular}

